# Baseball rules engine VI

May 24, 2024

Version V is my first pygame rules engine, which is text based. That got a bit heavy before I had all the functionality in place.
Spent May 21-24 re-designing this in other documents.

Now, want to implement the leanest possible foundation in Jupyter in order to layer on additional requirements.

## User input
I think I just call user_input( [item 1, item 2... ])

#### Type 1
- FBC: *['fbc']* >>>>> len = 1 | str

#### Type 2
- Runner tag out: *['tag r', 'isaac']* >>>>> len = 2 | str, str
- Base tag out: *['tag b', 1 or 2 or 3]* >>>>> len = 2 | str, int
- Change state: *['state', 0 or 1]* >>>>> len = 2 | str, int

#### Type 3
- Set goal: *['goal', 'isaac', 1 or 2 or 3 or 4]* >>>>> len = 3 | str, str, int
- Occupy base: *['occupy', 'isaac', 1 or 2 or 3 or 4]* >>>>> len = 3 | str, str, int

In [4]:
class RuEg:
    
    def __init__(self, runners):
        self.runners = runners # dict: {name: object}
        self.runners_out = []
        self.state = 0 

    
    def master_do(self, li):
        
        func_map = {'fbc': self.do_fbc, 'tag r': self.fielder_tags_runner, 'tag b': self.fielder_tags_base,
                   'state': self.change_state, 'goal': self.set_goal, 'occupy': self.occupy_attain_base}
        

        if li[0] not in func_map:
            print('Bad input')
            return

        func_map[ li[0] ](li)
       
        
    def do_fbc(self, li):
        print(li[1])
        
    def fielder_tags_runner(self, x):
        pass
    
    def fielder_tags_base(self, x):
        pass
    def change_state(self, x):
        pass
    
    def set_goal(self, x):
        pass
    
    def occupy_attain_base(self, x):
        pass

In [6]:
ru = RuEg('runners')

li = ['fbc', 'yo you ma']
ru.master_do(li)

yo you ma
